### PGCB Hourly Generation Dataset (Bangladesh)
This dataset, published by the Power Grid Company of Bangladesh (PGCB), provides hourly records of electricity generation, demand, and loadshedding across the national grid.<br>
Source: https://archive.ics.uci.edu/dataset/1175/pgcb+hourly+generation+dataset+(bangladesh)

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [35]:
df = pd.read_excel("PGCB_date_power_demand.xlsx")
df.head()

,datetime,generation_mw,demand_mw,load_shedding,gas,liquid_fuel,coal,hydro,solar,wind,india_bheramara_hvdc,india_tripura,india_adani,nepal,remarks
0,2015-04-19 22:00:00,6323.0,6323,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN
1,2015-04-19 21:00:00,6667.0,6667,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN
2,2015-04-19 19:00:00,6897.0,6897,0,4415,1836,161,41,NaN,NaN,444,0,NaN,NaN,NaN
3,2015-04-19 18:30:00,6933.0,6933,0,4423,1862,159,45,NaN,NaN,444,0,NaN,NaN,Evening_Peak
4,2015-04-19 18:00:00,6874.0,6874,0,4319,1892,155,65,NaN,NaN,443,0,NaN,NaN,NaN


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92650 entries, 0 to 92649
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              92650 non-null  datetime64[ns]
 1   generation_mw         92650 non-null  float64       
 2   demand_mw             92650 non-null  int64         
 3   load_shedding         92650 non-null  int64         
 4   gas                   92650 non-null  int64         
 5   liquid_fuel           92650 non-null  int64         
 6   coal                  92650 non-null  int64         
 7   hydro                 92650 non-null  int64         
 8   solar                 70517 non-null  float64       
 9   wind                  18676 non-null  float64       
 10  india_bheramara_hvdc  92650 non-null  int64         
 11  india_tripura         92650 non-null  int64         
 12  india_adani           7338 non-null   float64       
 13  nepal           

In [37]:
# Let's check the missing value
df.isnull().sum()

datetime                    0
generation_mw               0
demand_mw                   0
load_shedding               0
gas                         0
liquid_fuel                 0
coal                        0
hydro                       0
solar                   22133
wind                    73974
india_bheramara_hvdc        0
india_tripura               0
india_adani             85312
nepal                   87299
remarks                 86257
dtype: int64

In [38]:
df.shape

(92650, 15)

In [39]:
# Check duplicate rows
print(f"Duplicate rows: {df.duplicated().sum()}")

Duplicate rows: 160


In [40]:
df = df.drop_duplicates()
df.shape

(92490, 15)

In [41]:
print("Datetime min:",df['datetime'].min())
print("Datetime max:",df['datetime'].max())

Datetime min: 2015-04-19 00:00:00
Datetime max: 2025-06-17 12:00:00


In [42]:
# I have a lot of missing data and the ds is not sorted.
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)
df.head()

,datetime,generation_mw,demand_mw,load_shedding,gas,liquid_fuel,coal,hydro,solar,wind,india_bheramara_hvdc,india_tripura,india_adani,nepal,remarks
0,2015-04-19 00:00:00,4821.0,4821,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN
1,2015-04-19 01:00:00,3612.0,3612,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN
2,2015-04-19 02:00:00,3727.0,3727,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN
3,2015-04-19 03:00:00,3632.0,3632,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN
4,2015-04-19 04:00:00,3641.0,3641,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN


In [43]:
# Dropping columns with 75% missing columns 
columns_to_drop =  [column for column in df.columns if df[column].isnull().mean() > 0.75]
print(f"Dropping: {columns_to_drop}")
df = df.drop(columns=columns_to_drop)
df.shape

Dropping: ['wind', 'india_adani', 'nepal', 'remarks']


(92490, 11)

In [49]:
# Again let's check the misssing value
df.isnull().sum()


datetime                    0
generation_mw               0
demand_mw                   0
load_shedding               0
gas                         0
liquid_fuel                 0
coal                        0
hydro                       0
solar                   22023
india_bheramara_hvdc        0
india_tripura               0
dtype: int64

In [50]:
check_columns = ['generation_mw', 'demand_mw', 'load_shedding', 'gas', 'liquid_fuel', 'coal']
quantiles = df[check_columns].quantile([0.01, 0.25, 0.50, 0.75, 0.99, 1.0])
print(quantiles.to_string())

      generation_mw  demand_mw  load_shedding      gas  liquid_fuel      coal
0.01   4.182890e+03     4167.0           0.00   3311.0         0.00      0.00
0.25   6.817625e+03     6816.0           0.00   4420.0       720.00    159.00
0.50   8.416000e+03     8426.0           0.00   5024.0      1535.00    402.00
0.75   1.056000e+04    10636.0           0.00   5813.0      2462.00   1226.75
0.99   1.450011e+04    15150.0        1519.11   7099.0      5041.11   4556.00
1.00   6.452650e+07   156050.0       65359.00  74818.0  29222897.00  31687.00


In [51]:
num_cols = df.select_dtypes(include='number').columns.tolist()
for c in num_cols:
    lo, hi = df[c].quantile(0.01), df[c].quantile(0.99)
    df[c] = df[c].clip(lo, hi)

print("After clipping — max values:")
print(df[check_columns].max())

After clipping — max values:
generation_mw    14500.11
demand_mw        15150.00
load_shedding     1519.11
gas               7099.00
liquid_fuel       5041.11
coal              4556.00
dtype: float64


In [ ]:
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
print(df[['hour', 'day_of_week', 'month', 'year']].describe().round(2))

           hour  day_of_week     month      year
count  92490.00      92490.0  92490.00  92490.00
mean      11.77          3.0      6.53   2019.88
std        6.90          2.0      3.41      2.97
min        0.00          0.0      1.00   2015.00
25%        6.00          1.0      4.00   2017.00
50%       12.00          3.0      6.00   2020.00
75%       18.00          5.0      9.00   2022.00
max       23.00          6.0     12.00   2025.00


In [54]:
# Ratio features: generation/demand, gas share, total imports
df['gen_demand_ratio'] = df['generation_mw'] / df['demand_mw'].replace(0, np.nan)
df['gas_share'] = df['gas'] / df['generation_mw'].replace(0, np.nan)
df['total_imports'] = df['india_bheramara_hvdc'] + df['india_tripura']
# Fill potential inf/nan from division
df[['gen_demand_ratio', 'gas_share']] = df[['gen_demand_ratio', 'gas_share']].replace([np.inf, -np.inf], np.nan).fillna(0)
print(df[['gen_demand_ratio', 'gas_share', 'total_imports']].describe().round(4))

       gen_demand_ratio   gas_share  total_imports
count        92490.0000  92490.0000     92490.0000
mean             0.9944      0.6139       756.1218
std              0.0454      0.1258       252.9313
min              0.3540      0.2283       129.8900
25%              1.0000      0.5190       550.0000
50%              1.0000      0.6077       803.0000
75%              1.0000      0.7044      1001.0000
max              3.3285      1.6972      1127.0000
